## Latent Dirichlet Allocation ##
**Watch these videos:**
- https://www.youtube.com/watch?v=azXCzI57Yfc
- https://www.youtube.com/watch?v=T05t-SqKArY
- https://www.youtube.com/watch?v=BaM1uiCpj_E

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [ ]:
'''
Load the dataset from the CSV and save it to 'data_text'. We only need the'headline_text' column,
so drop the rest of the columns. Call the final dataframe "documents" 
Then, add an 'index' column to the dataframe. Be sure to name it index.
Hint: dataframe.index
'''
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/abcnews-date-text.csv')
data_text = data[:300000][['headline_text']]
data_text['index'] = data_text.index
documents = data_text

print(documents['index'])

0              0
1              1
2              2
3              3
4              4
           ...  
299995    299995
299996    299996
299997    299997
299998    299998
299999    299999
Name: index, Length: 300000, dtype: int64


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Let's glance at the dataset:

In [ ]:
'''
print the total number of documents
'''
print(len(documents))

300000


In [ ]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.

This time, we'll make use of the gensim library and show you how  to preprocess text easily. 

In [ ]:
'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Lemmatizer Example
Before preprocessing our dataset, let's first look at an lemmatizing example. What would be the output if we lemmatized the word 'went':

In [ ]:
print(WordNetLemmatizer().lemmatize('went', pos = 'v')) # past tense to present tense


go


### Stemmer Example
Let's also look at a stemming example. Let's throw a number of words at the stemmer and see how it deals with each one:

In [ ]:
stemmer = SnowballStemmer("english")
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]

pd.DataFrame(data={'original word':original_words, 'stemmed':singles })

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [ ]:
'''
Write a function to perform the pre processing steps on the entire dataset
'''
"""def lemmatize_stemming(text):
    '''
        1) Lemmatize the text
        2) Then, stem the text
        3) return final text
    '''
    pass



# Tokenize and lemmatize. Look up how preprocessing is being done
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
             result.append(lemmatize_stemming(token))
        
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
        #pass
        
        
    return result"""



stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and Lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. To do that, let's use the [map](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) function from pandas to apply `preprocess()` to the `headline_text` column

**Note**: This may take a few minutes

In [ ]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents['headline_text'].map(preprocess)

In [ ]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
#  Todo
dictionary = gensim.corpora.Dictionary(processed_docs)

In [ ]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [ ]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
- keep 100000 most frequent tokens
'''
# TODO: apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'.
Hint: Since we want a BoW model for each document, you can use list comprehension instead of for loops. List
comprehension is the more pythonic way.
'''
#  Todo:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num]

[(71, 1), (107, 1), (462, 1), (3530, 1)]

In [ ]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [ ]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models
# TODO
tfidf = models.TfidfModel(bow_corpus)


In [ ]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
# TODO
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [ ]:
# LDA multicore 
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'

If Multicore is throwing errors on you laptop, use LDA mono-core, gensim.models.LdaModel
'''
# TODO

lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.022*"closer" + 0.021*"test" + 0.020*"lead" + 0.017*"talk" + 0.014*"south" + 0.013*"law" + 0.012*"take" + 0.012*"timor" + 0.010*"clash" + 0.010*"west"


Topic: 1 
Words: 0.091*"polic" + 0.028*"seek" + 0.025*"investig" + 0.022*"miss" + 0.016*"search" + 0.015*"probe" + 0.013*"region" + 0.011*"offic" + 0.011*"bodi" + 0.011*"shoot"


Topic: 2 
Words: 0.016*"record" + 0.014*"break" + 0.014*"australia" + 0.014*"look" + 0.012*"drought" + 0.012*"rain" + 0.012*"price" + 0.012*"dead" + 0.010*"crew" + 0.009*"fall"


Topic: 3 
Words: 0.051*"water" + 0.032*"warn" + 0.018*"urg" + 0.015*"industri" + 0.014*"continu" + 0.013*"farmer" + 0.012*"busi" + 0.012*"begin" + 0.011*"worker" + 0.010*"threat"


Topic: 4 
Words: 0.016*"elect" + 0.016*"iraq" + 0.014*"howard" + 0.013*"labor" + 0.013*"market" + 0.013*"deal" + 0.013*"reject" + 0.012*"council" + 0.012*"say" + 0.012*"appeal"


Topic: 5 
Words: 0.040*"charg" + 0.035*"court" + 0.033*"face" + 0.022*"kill" + 0.020*"murder" + 0.020*"accus" +

### Classification of the topics ###

Using the words in each topic and their corresponding weights, what categories were you able to infer?

* 0: 
* 1: 
* 2: 
* 3: 
* 4: 
* 5: 
* 6: 
* 7:  
* 8: 
* 9: 

## Step 4.2 Running LDA using TF-IDF ##

In [ ]:
'''
Define lda model using corpus_tfidf
'''
# TODO
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [ ]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.013*"rudd" + 0.008*"solomon" + 0.008*"liber" + 0.006*"south" + 0.006*"honour" + 0.005*"ralli" + 0.005*"pipelin" + 0.005*"poll" + 0.005*"resourc" + 0.005*"pulp"


Topic: 1 Word: 0.008*"coast" + 0.007*"price" + 0.006*"gold" + 0.006*"nurs" + 0.006*"toll" + 0.006*"staff" + 0.006*"rise" + 0.006*"govt" + 0.006*"rais" + 0.006*"qanta"


Topic: 2 Word: 0.016*"kill" + 0.013*"iraq" + 0.013*"arrest" + 0.011*"bomb" + 0.008*"firefight" + 0.008*"troop" + 0.007*"polic" + 0.007*"blast" + 0.007*"cyclon" + 0.006*"attack"


Topic: 3 Word: 0.009*"england" + 0.008*"plead" + 0.007*"guilti" + 0.007*"uranium" + 0.007*"season" + 0.006*"elect" + 0.006*"blue" + 0.006*"dismiss" + 0.006*"fin" + 0.006*"comment"


Topic: 4 Word: 0.012*"hick" + 0.007*"israel" + 0.007*"iemma" + 0.006*"isra" + 0.006*"restrict" + 0.006*"palestinian" + 0.006*"afghanistan" + 0.006*"eas" + 0.006*"coal" + 0.006*"withdraw"


Topic: 5 Word: 0.021*"charg" + 0.018*"court" + 0.018*"polic" + 0.015*"murder" + 0.013*"search" + 0.011

### Classification of the topics ###

As we can see, when using tf-idf, heavier weights are given to words that are not as frequent which results in nouns being factored in. That makes it harder to figure out the categories as nouns can be hard to categorize. This goes to show that the models we apply depend on the type of corpus of text we are dealing with. 

Using the words in each topic and their corresponding weights, what categories could you find?

* 0: 
* 1:  
* 2: 
* 3: 
* 4:  
* 5: 
* 6: 
* 7: 
* 8: 
* 9: 

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model

We will check to see where our test document would be classified. 

In [ ]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [ ]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.48661112785339355	 
Topic: 0.016*"record" + 0.014*"break" + 0.014*"australia" + 0.014*"look" + 0.012*"drought" + 0.012*"rain" + 0.012*"price" + 0.012*"dead" + 0.010*"crew" + 0.009*"fall"

Score: 0.35336190462112427	 
Topic: 0.091*"polic" + 0.028*"seek" + 0.025*"investig" + 0.022*"miss" + 0.016*"search" + 0.015*"probe" + 0.013*"region" + 0.011*"offic" + 0.011*"bodi" + 0.011*"shoot"

Score: 0.020010266453027725	 
Topic: 0.051*"water" + 0.032*"warn" + 0.018*"urg" + 0.015*"industri" + 0.014*"continu" + 0.013*"farmer" + 0.012*"busi" + 0.012*"begin" + 0.011*"worker" + 0.010*"threat"

Score: 0.020006544888019562	 
Topic: 0.018*"return" + 0.017*"hold" + 0.014*"question" + 0.014*"work" + 0.014*"resid" + 0.012*"firefight" + 0.011*"blaze" + 0.010*"rais" + 0.010*"unit" + 0.010*"battl"

Score: 0.020003756508231163	 
Topic: 0.061*"govt" + 0.029*"council" + 0.024*"fund" + 0.022*"plan" + 0.017*"urg" + 0.017*"boost" + 0.013*"servic" + 0.012*"rise" + 0.012*"defend" + 0.011*"health"

Score: 0.0

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model

In [ ]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
Code would be similar to above
'''
# Todo
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.819951057434082	 
Topic: 0.012*"hick" + 0.007*"israel" + 0.007*"iemma" + 0.006*"isra" + 0.006*"restrict" + 0.006*"palestinian" + 0.006*"afghanistan" + 0.006*"eas" + 0.006*"coal" + 0.006*"withdraw"

Score: 0.020011229440569878	 
Topic: 0.011*"govt" + 0.010*"health" + 0.008*"fund" + 0.008*"water" + 0.007*"indigen" + 0.007*"urg" + 0.007*"servic" + 0.007*"council" + 0.006*"plan" + 0.006*"farm"

Score: 0.020007509738206863	 
Topic: 0.016*"kill" + 0.013*"iraq" + 0.013*"arrest" + 0.011*"bomb" + 0.008*"firefight" + 0.008*"troop" + 0.007*"polic" + 0.007*"blast" + 0.007*"cyclon" + 0.006*"attack"

Score: 0.020006822422146797	 
Topic: 0.013*"drought" + 0.009*"govt" + 0.008*"plan" + 0.008*"nuclear" + 0.008*"water" + 0.008*"council" + 0.007*"rate" + 0.006*"farmer" + 0.006*"rat" + 0.006*"worri"

Score: 0.02000528946518898	 
Topic: 0.040*"closer" + 0.029*"crash" + 0.013*"die" + 0.010*"fatal" + 0.009*"polic" + 0.009*"road" + 0.009*"accid" + 0.008*"plane" + 0.008*"investig" + 0.007*"highway"



## Step 6: Testing model on unseen document ##

In [ ]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.4200000762939453	 Topic: 0.022*"closer" + 0.021*"test" + 0.020*"lead" + 0.017*"talk" + 0.014*"south"
Score: 0.2199999839067459	 Topic: 0.018*"return" + 0.017*"hold" + 0.014*"question" + 0.014*"work" + 0.014*"resid"
Score: 0.21999235451221466	 Topic: 0.040*"charg" + 0.035*"court" + 0.033*"face" + 0.022*"kill" + 0.020*"murder"
Score: 0.020003903657197952	 Topic: 0.038*"crash" + 0.025*"jail" + 0.021*"road" + 0.017*"die" + 0.016*"death"
Score: 0.02000369131565094	 Topic: 0.036*"report" + 0.024*"opposit" + 0.023*"power" + 0.014*"win" + 0.013*"final"
Score: 0.019999999552965164	 Topic: 0.091*"polic" + 0.028*"seek" + 0.025*"investig" + 0.022*"miss" + 0.016*"search"
Score: 0.019999999552965164	 Topic: 0.016*"record" + 0.014*"break" + 0.014*"australia" + 0.014*"look" + 0.012*"drought"
Score: 0.019999999552965164	 Topic: 0.051*"water" + 0.032*"warn" + 0.018*"urg" + 0.015*"industri" + 0.014*"continu"
Score: 0.019999999552965164	 Topic: 0.016*"elect" + 0.016*"iraq" + 0.014*"howard" + 0.01

### Now, look up different ways we can transform our words or use different techniques or improvements that we can make to make our Topic Classifier more accurate. Write them down below:

**We can:**
- Append points here
- 